# Summary

__How to solve the tasks and why that way are discussed here.__

Physical unit of entries is not mentioned in the data set, and is presumed to be kiloWatt (kW). The physical unit only matters in interpretation because forecasted values are more or less the same.

The distribution of missing entries can be summarised as:

- For households 162, 428 and 432, two entries in different sets of dates are missed".
- For all the other households, 48 entries in different sets of dates are missed.
- There are only 186 time points when data is complete.

Linear interpolation for such missing entries is not very useful.


## 1. Clustering

There are at least two challenges:

- Too many missing days distribute differently among units.
- All series are non-stationary because of non-business-days, long-term trends, and seasonality.

### Why not k-Means

Missing entries must be filled with meaningfull values first, or algorithms, like k-Means do not work. Clearly, neither to fill with 0 nor to interpolate is a good choice in terms of time series. 

Monte Carlo simulations can be used, but the resulted effect is not known. The argument is that most clustering algorithms rely on calculation of cluster centres, which involves multiple units at the same time. Nevertheless, the joint set of missing days from those units might be large compared to that from pair-wise units, so simulated entries exist in more time index, distorting cluster centres.

Moreover, most methods to validate cluster results are based on original data, so they cannot be used, either. Probably the only option left is to validate based on pair-wise similarity measures, like Euclidean distance and Pearson correlation.

### Why not Agglomerative Hierarchical (Single Linkage)

Agglomerative hierarchical clustering (AHC) using single linkage relies on some distance matrix resulted from original data only. Entries in such distance matrix measure the similarity of some unit pair according to a pre-defined criterion. For similarity between time series, Pearson correlation, instead of Euclidean distance, is usually used.

When number of clusters is set to 4, for example, differences in sizes -- 497, 1, 1, 1 respectively -- are significant. The reason is that units in last three clusters do not have strong correlation with any other unit. That is, they are outliers. Though their distribution among clusters does not have a huge impact, they prevent the algorithm to process units in the first big cluster.

### My Approach: Max Spanning Tree on Distance Matrix

AHC using single linkage is an optimisation problem in essence. To run max spanning tree on distance matrix yields better result, actually. Another advantage of this approach is that the complete graph (with lightgrey-coloured edges in the following figure), which represesnts the distance matrix, provides a more intuitive way to choose clusters.

Units 70 ~ 99 are considered in the following example, resulting a clearer figure:

- The max spanning tree of the complete graph is highlighted by black edges. 
- 77 and 79 have high degrees, and can be seen as representative units for two clusters. 
- It seems to be a good idea to have another cluster, represented by unit 99.
- Weakest links between such three units (two blue edges) are found. If they are removed, the tree becomes a forest with three components, which correspond to three clusters.

<img src="./img/MST.png" width="400">

### How to Examine Clustering Results

The distance matrix can be sorted according to clusters. Correlations between members within clusters are supposed to have higher values. In constrast, off-diagonal block matrices represent inter-cluster correlations, which should be low and even negative.

## 2. Regression

Methods for two sets of forecasts, using hourly and daily profiles respectively, are discussed.

### Why not Multivariate Time Series

It is known that there is no relationship between profiles from different households, because their behaviours are not directly correlated. The only regressor considered should be time. Others like temperature may contribute, but are unknown for now.

So it is not necessary to use vector auto regression (VAR). Profiles can be modelled independently.

### Why Facebook `Prophet`

[`Prophet`](https://facebook.github.io/prophet/) is an open-source package for modelling and forecasting univariate time series by Facebook. Its model structure is similar to that of generalised additive models (GAM), where distinct non-linear terms are integrated in a linear way. `Prophet` works best when the series shows strong and multiple seasonality, and it is robust to missing values and effect from holidays, making it the perfect choice in our case.

### Seasonal ARIMA for Daily Profiles

It is impossible to plot ACF and PACF for original series because of missing entries. As discussed before, they are adjacent so it is hard to fill with meaningful values. Result from `Prophet` provides a starting point. To have a weekly season is a good idea, and it should be "integrated", in order to remove the long-term trend. An __integrated SARIMA (with AR1, MA1 and weekly seasonal AR1) structure__ is used for all households.

### Why not Seasonal ARIMA for Hourly Profiles

There are daily and weekly seasonal components, which cannot be modelled by any seasonal ARIMA at the same time.

### Model Validation

There are systematic ways for validation. It is challenging to conduct a cross validation for time series models, especially when there is long-term trend. __Pseudo out-of-sample validation__ is used instead. Here is a [description from `statsmodels`](https://www.statsmodels.org/stable/examples/notebooks/generated/statespace_forecasting.html#Cross-validation): "A common use case is to cross-validate forecasting methods by performing h-step-ahead forecasts recursively".

### Results

Two sets of forecasts for 500 households are made:

- 3-step forecasts based on daily profile, done by integrated SARIMA (with AR1, MA1 and weekly seasonal AR1), in [a comma-separated CSV file](https://github.com/edxu96/elec_consumption/blob/master/results/three_step_daily.csv)
- 4-step forecasts based on hourly profile, done by Facebook `Prophet`, in [another comma-separated CSV file](https://github.com/edxu96/elec_consumption/blob/master/results/four_step_hourly.csv)